### In this Notebook we will fetch team information data from data lake folder and insert it into Azure Sql-Server RD database.

In [0]:
# !pip install tqdm # if you face issue related to tqdm
import pyodbc
import pandas as pd
from tqdm import tqdm

DATA_LAKE_FOLDER_PATH = "E:/Google_Drive_Contents/data/Final_data_files/"

# fill the database cred here
SERVER = '*'
DATABASE = '*'
USERNAME = '*'
PASSWORD = '*'

In [0]:
def get_azure_database_connection(server,database,username,password):
    """
    This function will return an aws conn object for azure SQl Server database.
    return : AWS connection object
    """
    cnxn = None
    try:
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
        return cnxn
        
    except Exception as e:
        print(e)

In [0]:
team_data = pd.read_csv(DATA_LAKE_FOLDER_PATH+'/team_captain_owner_information.csv')

In [0]:
team_data = team_data[['Team','City','Home_ground','Debut','Owner','Captain','Dissolved']]
team_data.head(2)

,Team,City,Home_ground,Debut,Owner,Captain,Dissolved
0,Chennai Super Kings,"Chennai, Tamil Nadu",M. A. Chidambaram Stadium,2008,India Cements,MS Dhoni,0
1,Delhi Capitals,"Delhi, NCR",Arun Jaitley Stadium,2008,GMR Group and JSW Group,Shreyas Iyer,0


In [0]:
def insert_data_into_database(db_connection,df):
    """
    This function will insert data into the database row by row.
    parameters :
    db_connection : database connection object
    df : pandas DataFrame which has data
    return : None
    """
    try:
        
         # get the cursor
        cur = db_connection.cursor()
        for i,row in tqdm(df.iterrows(),total = df.shape[0]):
            row_list = list(row) # for easy access
            # create insert statment
            Insert_Statment = f"""Insert into ipl_teams(team_name,team_city,home_ground,debut_year,owner_name,captain_name,dissolved_year,captain_id) 
                values('{row_list[0]}','{row_list[1]}','{row_list[2]}',{row_list[3]},'{row_list[4]}','{row_list[5]}',{row_list[6]},{0})"""
            
#             print(Insert_Statment)

            
            # execute the query
            cur.execute(Insert_Statment)
            # commit the connection
            db_connection.commit()


    except Exception as e:
        print(e)

In [0]:
db_connection = get_azure_database_connection(SERVER,DATABASE,USERNAME,PASSWORD)
insert_data_into_database(db_connection,team_data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.72it/s]


In [0]:
db_connection.close()

In [0]:
team_data.shape

(12, 7)